# import Jason file

In [1]:
#standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#Additional Imports
import os,json,math,time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
import json
with open('/Users/rowenahan/.secret/yelp_api.json') as f: #change the path to match YOUR path!!
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [3]:
from yelpapi import YelpAPI
yelp_api = YelpAPI(login["api-key"],timeout_s=5.0)
yelp_api

## Search location and term

In [4]:
LOCATION = "Oahu, Hawaii"
TERM = "Pizza"

### Checking to see if Json file exists

In [5]:
#Specifying JSON_FILE filename (can include a folder
#includ the search term in the filename)
JSON_FILE = "Data/results_in_progress_HNL_pizza.json"
JSON_FILE

'Data/results_in_progress_HNL_pizza.json'

In [6]:
# Check if JSON_FILE exists
file_exists=os.path.isfile(JSON_FILE)
#If it does not exist:
if file_exists==False:
    #Create any needed folders
    #get the folder Name only
    folder=os.path.dirname(JSON_FILE)
    #IF JSON_FILE included a folder
    if len(folder)>0:
        #create the folder
        os.makedirs(folder,exist_ok=True)

    #INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file")

    #save an empty list
    with open(JSON_FILE, "w") as f:
        json.dump([],f)
#if it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_HNL_pizza.json already exists.


In [7]:
#Load previous results and use len of results for offset
with open(JSON_FILE,"r") as f:
    previous_results = json.load(f)
#Set offset based on previous results
n_results = len(previous_results)
print(f"- {n_results} previous resuts found.")

- 0 previous resuts found.


In [8]:
search_results = yelp_api.search_query(location = LOCATION,
                                      term= TERM,
                                      offset=n_results)
print(type(search_results))
search_results.keys()

<class 'dict'>


dict_keys(['businesses', 'total', 'region'])

In [9]:
search_results["total"]

925

In [10]:
#How many did we get the details for?
results_per_page = len(search_results["businesses"])
results_per_page

20

In [11]:
#import additional packages for controlling our loop
import time, math
#Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((search_results["total"]-n_results)/results_per_page)
n_pages

47

In [12]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [13]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    #adds 200 ms pause
    time.sleep(.2)

  0%|          | 0/47 [00:00<?, ?it/s]

In [14]:
os.remove(JSON_FILE)
os.path.isfile(JSON_FILE)

False

In [15]:
def create_json_file(JSON_FILE, delete_if_exists=False):
        #check if JSON_FILE exists
        file_exists=os.path.isfile(JSON_FILE)
        # if it DOSE exist:
        if file_exists == True:
            #check if user wants to delete if exists
            if delete_if_exists==True:
                print(f"[!]  {JSON_FILE} already exists. Deleting previous file...")
                #delet file and confirm it no longer exits
                os.remove(JSON_FILE)
                # recursive call to function after old file deleted
                create_json_file(JSON_FILE, delete_if_exists=False)
            else:
                print(f"[i] {JSON_FILE} already exists.")
        #if it dose NOT exist:
        else:
            #infor user and save empty list
            print(f"[i] {JSON_FILE} not found. Sving empty list to new file.")
            # Create any needed folders
            #Get the folder name only
            folder=os.path.dirname(JSON_FILE)
            #if JSON_FILE included a folder
            if len(folder)>0:
                #create the folder
                os.makedirs(folder,exist_ok=True)
            #save empty list to start the json file
            with open(JSON_FILE,"w") as f:
                json.dump([],f)

In [16]:
# Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE,delete_if_exists=True)
# loadprevious results and use en of results for offset
with open(JSON_FILE,"r") as f:
    previous_results=json.load(f)
#set offset based on previous results
n_results=len(previous_results)
print(f"- {n_results} previoous results found.")
#use our yelp_api variable"s search_query method to perform our API call
results=yelp_api.search_query(location=LOCATION,
                             term=TERM,
                             offset=n_results)
#how many results total?
total_results = results["total"]
#how many did we get the details for?
resutls_per_page = len(results["businesses"])
#use math ceil to round up for the total number of pages of results
n_pages = math.ceil((results["total"]-n_results)/results_per_page)
n_pages

[i] Data/results_in_progress_HNL_pizza.json not found. Sving empty list to new file.
- 0 previoous results found.


47

In [17]:
for i in tqdm_notebook(range(1,n_pages+1)):
    #Read in results in progress file and check the length
    with open(JSON_FILE,"r") as f:
        previous_results= json.load(f)
     #same number of results for the use as offset
    n_results=len(previous_results)
    if (n_results + results_per_page)>1000:
        print("Exceeded 1000 api calls. Stopping loop.")
        break

    #use n_results as the offset
    results=yelp_api.search_query(location=LOCATION,
                                 term=TERM,
                                 offset=n_results)
    #append new results and save to file
    previous_results.extend(results["businesses"])
    #display (previous_results)
    with open(JSON_FILE,"w") as f:
        json.dump(previous_results,f)

    time.sleep(.2)

  0%|          | 0/47 [00:00<?, ?it/s]

In [18]:
#load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,b7beu_ddBb7yiMc0Tgfr0A,old-town-pizzeria-and-tap-house-wahiawa,Old Town Pizzeria & Tap House,https://s3-media3.fl.yelpcdn.com/bphoto/TYq3l6...,False,https://www.yelp.com/biz/old-town-pizzeria-and...,39,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 21.4935665206445, 'longitude': -1...",[],$$,"{'address1': '167 S Kamehameha Hwy', 'address2...",+18082005800,(808) 200-5800,6806.093112
1,ev-HilLF1ru2S76NkAxQIA,uncles-ny-deli-pizza-pasta-and-subs-wahiawa,"Uncle's NY Deli, Pizza, Pasta & Subs",https://s3-media4.fl.yelpcdn.com/bphoto/myqY1l...,False,https://www.yelp.com/biz/uncles-ny-deli-pizza-...,334,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 21.49506, 'longitude': -158.05007}",[pickup],$$,"{'address1': '1718 Wilikina Dr', 'address2': '...",+18085173232,(808) 517-3232,8978.617509
2,G36RP1LZPZDpBIM0SRP5UA,bostons-north-end-pizza-bakery-aiea-3,Boston's North End Pizza Bakery,https://s3-media2.fl.yelpcdn.com/bphoto/G1PY2j...,False,https://www.yelp.com/biz/bostons-north-end-piz...,741,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 21.380100434609627, 'longitude': ...","[pickup, delivery]",$$,"{'address1': '98-302 Kamehameha Hwy', 'address...",+18084874055,(808) 487-4055,11779.251057
3,_fb2LPaNzHexi7tPQhcnXg,da-big-kahunas-pizza-honolulu,Da Big Kahuna's Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/5laNR4...,False,https://www.yelp.com/biz/da-big-kahunas-pizza-...,1277,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 21.335218, 'longitude': -157.91598}",[delivery],$$,"{'address1': '550 Paiea St', 'address2': '', '...",+18088335588,(808) 833-5588,17198.356628
4,2qoHbQ6YzCJS5Xmpom4opA,little-g-cafe-by-garibaldis-honolulu,Little G Cafe By Garibaldi's,https://s3-media4.fl.yelpcdn.com/bphoto/9jSkrN...,False,https://www.yelp.com/biz/little-g-cafe-by-gari...,85,"[{'alias': 'gourmet', 'title': 'Specialty Food...",4.5,"{'latitude': 21.296945950477628, 'longitude': ...",[delivery],$$,"{'address1': '808 Sheridan St', 'address2': ''...",+18086999116,(808) 699-9116,24244.559429


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
920,a1TTKkQqmdVH9-rOxButIA,panda-express-honolulu-10,Panda Express,https://s3-media3.fl.yelpcdn.com/bphoto/YBVyCz...,False,https://www.yelp.com/biz/panda-express-honolul...,227,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",3.0,"{'latitude': 21.29726, 'longitude': -157.83767}","[delivery, pickup]",$$,"{'address1': '1505 S. King St.', 'address2': '...",+18089476891,(808) 947-6891,24477.754276
921,1F_zEDwhtq9SfBX429pA1w,subway-honolulu-119,Subway,https://s3-media4.fl.yelpcdn.com/bphoto/Zu5Ofi...,False,https://www.yelp.com/biz/subway-honolulu-119?a...,55,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",2.0,"{'latitude': 21.2749824032138, 'longitude': -1...","[delivery, pickup]",$,"{'address1': '4614 Kilauea Ave', 'address2': '...",+18087342240,(808) 734-2240,29633.076538
922,JGxJXf4DrzyF4b36oiQEtQ,panda-express-honolulu-3,Panda Express,https://s3-media2.fl.yelpcdn.com/bphoto/bxa8BD...,False,https://www.yelp.com/biz/panda-express-honolul...,104,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",3.0,"{'latitude': 21.27763, 'longitude': -157.78688}","[delivery, pickup]",$$,"{'address1': '4211 Waialae Ave. A1&a2', 'addre...",+18087395098,(808) 739-5098,29442.362879
923,Ya8dWhNcGI5WAkXKXmOLkA,barnes-and-noble-cafe-honolulu-2,Barnes & Noble Cafe,https://s3-media2.fl.yelpcdn.com/bphoto/snp-u7...,False,https://www.yelp.com/biz/barnes-and-noble-cafe...,54,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",2.5,"{'latitude': 21.2918195, 'longitude': -157.846...",[delivery],$,"{'address1': '1450 Ala Moana Blvd', 'address2'...",+18089497307,(808) 949-7307,24548.681970
924,coOKvZmB0CBWOgeYOaRPVQ,buffalo-wild-wings-honolulu-5,Buffalo Wild Wings,https://s3-media2.fl.yelpcdn.com/bphoto/ojj0FT...,False,https://www.yelp.com/biz/buffalo-wild-wings-ho...,318,"[{'alias': 'tradamerican', 'title': 'American ...",2.0,"{'latitude': 21.2913, 'longitude': -157.84338}","[delivery, pickup]",$$,"{'address1': '1450 Ala Moana Blvd', 'address2'...",+18089425445,(808) 942-5445,24736.761099


In [19]:
#final_df.duplicated().sum()

TypeError: unhashable type: 'list'

In [20]:
final_df.duplicated(subset="id").sum()

0

In [21]:
final_df=final_df.drop_duplicates(subset="id")
final_df.duplicated(subset="id").sum()

0

In [23]:
final_df.to_csv("Data/final_results_HNL_pizza.csv.gz",
               compression="gzip",index=False)

### Import pandas

In [ ]:
biz=pd.DataFrame(search_results["businesses"])
biz.head(2)